In [1]:
import pandas as pd
import os
import pprint


In [30]:
data_dir = '../data'

oilspills_csvpath = os.path.join(data_dir, 'US_oilspills.csv')


Initial findings from loading data:
-  3710 rows of data, each has an ID, open_date, name, lat and lon. 
-  We drop all the columns with all NaN values as these contain no useful information
-  We also drop the row of index '8039' as this is the only one with data inside the other 'field' columns (12/13/14/15/24/25 with 1 non-null value each) and is an outlier. This may infact contain useful information however we are unsure what to do with it and want to visualise the bulk of our data with a simple version 1 right now. This could be a future endeavour
-  It is unsure what field_10 represents, this could be important and a future task could be to include it. However, for now we remove it as it has no meaning right now.
-  It looks like field_11 contains furthe information. So we will rename the column as such.

So we create a function to load and format our data with the above points in mind.
We would be left with columns:
-  open_date 
-  name 
-  location
-  lat
-  lon
-  threat
-  commodity
-  max_ptl_release_gallons
-  description

In [41]:
def load_oilspills(csvpath: str) -> pd.DataFrame:
    
    oilspills = pd.read_csv(csvpath, 
                            index_col='id', 
                            dtype=dtypes, 
                            parse_dates=['open_date'])

    # remove anomalous row (contains additional fields in columns)
    row_8039 = oilspills[oilspills['field_12'].notna()]
    oilspills = oilspills.drop(row_8039.index)

    # drop columns with NaN values
    oilspills = oilspills.dropna(axis=1, how='all')

    # drop column: 'field_10'
    oilspills = oilspills.drop(['field_10'], axis=1)

    oilspills = oilspills.rename(columns={'field_11':'description'})
    
    return oilspills


In [43]:
load_oilspills(oilspills_csvpath).columns

Index(['open_date', 'name', 'location', 'lat', 'lon', 'threat', 'commodity',
       'max_ptl_release_gallons', 'description'],
      dtype='object')

field numbers: 12,13,14,15,24 and 25 have 1 non-null value each. Let's look at these.

In [ ]:
oilspills['threat'].value_counts()

In [ ]:
oilspills['commodity'].value_counts()
# seems like we have some cases where upper case is the same as lower case. 